<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-and-Define-Utils" data-toc-modified-id="Load-and-Define-Utils-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load and Define Utils</a></span><ul class="toc-item"><li><span><a href="#Magics" data-toc-modified-id="Magics-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Magics</a></span></li><li><span><a href="#Easy-Widget" data-toc-modified-id="Easy-Widget-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Easy Widget</a></span></li></ul></li><li><span><a href="#Data" data-toc-modified-id="Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data</a></span><ul class="toc-item"><li><span><a href="#Data-Source" data-toc-modified-id="Data-Source-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Data Source</a></span></li><li><span><a href="#Data-Convert" data-toc-modified-id="Data-Convert-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Data Convert</a></span></li><li><span><a href="#Data-Sink" data-toc-modified-id="Data-Sink-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Data Sink</a></span></li><li><span><a href="#Test" data-toc-modified-id="Test-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Test</a></span></li></ul></li><li><span><a href="#JetRep" data-toc-modified-id="JetRep-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>JetRep</a></span><ul class="toc-item"><li><span><a href="#Inference-Process" data-toc-modified-id="Inference-Process-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Inference Process</a></span></li><li><span><a href="#Pre-Worker" data-toc-modified-id="Pre-Worker-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Pre Worker</a></span></li><li><span><a href="#Post-Worker" data-toc-modified-id="Post-Worker-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Post Worker</a></span></li><li><span><a href="#Process-Class" data-toc-modified-id="Process-Class-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Process Class</a></span></li><li><span><a href="#Control-Class" data-toc-modified-id="Control-Class-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Control Class</a></span></li></ul></li><li><span><a href="#Repnet-Model" data-toc-modified-id="Repnet-Model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Repnet Model</a></span><ul class="toc-item"><li><span><a href="#Pre-Process" data-toc-modified-id="Pre-Process-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Pre Process</a></span></li><li><span><a href="#Model-Output" data-toc-modified-id="Model-Output-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Model Output</a></span></li><li><span><a href="#Post-Process" data-toc-modified-id="Post-Process-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Post Process</a></span><ul class="toc-item"><li><span><a href="#Calculate-Scores" data-toc-modified-id="Calculate-Scores-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Calculate Scores</a></span></li><li><span><a href="#Calculate-Count" data-toc-modified-id="Calculate-Count-4.3.2"><span class="toc-item-num">4.3.2&nbsp;&nbsp;</span>Calculate Count</a></span></li></ul></li></ul></li><li><span><a href="#Make-it-together" data-toc-modified-id="Make-it-together-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Make it together</a></span></li><li><span><a href="#References" data-toc-modified-id="References-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>References</a></span></li><li><span><a href="#FAQs" data-toc-modified-id="FAQs-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>FAQs</a></span></li></ul></div>

<div class="alert alert-warning">
<center>The Notebook Only Run On Jetson The Device</center>
</div>

In [1]:
notename = !uname -n
if 'nano' != notename[0]:
    raise RuntimeError('notebook server is not jetson device')

## Load and Define Utils

In [2]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard,tflite
%watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
%matplotlib inline
%config InlineBackend.figure_format='retina'
# %config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, IFrame, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
# display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, logging, time, random, math
import json, base64, requests, shutil
import os.path as osp
import numpy as np
import argparse, shlex, signal
import threading, queue

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x):
    try:
        x = x.strip()
        if x[0] == '/':
            with open(x) as fr:
                x = fr.read()
        elif 'github' in x or 'gitee' in x:
            if x.startswith('import '):
                x = x[7:]
            if x.startswith('https://'):
                x = x[8:]
            if not x.endswith('.py'):
                x = x + '.py'
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                uri = 'https://' + x
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                uri = x.replace('github.com', 'raw.githubusercontent.com')
                mod = uri.split('/')
                for s in ['main', 'master']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except:
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)
    
def get_logger(name, level=logging.DEBUG, filepath=None, console=True):
    logger = logging.getLogger(name)
    logger.setLevel(level)
    #  %(filename)s
    formatter = logging.Formatter('%(asctime)s - %(funcName)s:%(lineno)d - %(name)s - %(levelname)s - %(message)s')
    if console:
        console = logging.StreamHandler()
        console.setLevel(level)
        console.setFormatter(formatter)
        logger.addHandler(console)
    if filepath:
        filelog = logging.FileHandler(filepath)
        filelog.setLevel(level)
        filelog.setFormatter(formatter)
        logger.addHandler(filelog)
    return logger

import cv2

SCRIPT_DIR = '/home/nano/omega/jetson/scripts'
logger = get_logger('JetRep')

numpy 1.19.4
sklearn not installed
pandas 0.22.0
ipywidgets 6.0.0
cv2 4.1.1
PIL 5.1.0
matplotlib 2.1.1
plotly not installed
netron not installed
torch not installed
torchvision not installed
torchaudio not installed
tensorflow 2.5.0+nv21.8
tensorboard 2.6.0
tflite not installed
onnx 1.10.1
tf2onnx not installed
onnxruntime 1.8.1
tensorrt 8.0.1.6
tvm not installed


### Magics

In [3]:
@register_line_magic
def killall(line):
    pid_list=!ps -eo pid,command | grep $line  | grep -v 'grep' | cut -c1-6
    for pid in pid_list:
        pid = pid.strip()
        !kill -9 $pid

### Easy Widget

In [4]:
_IMPORT('https://gitee.com/qrsforever/nb_easy/easy_widget.py')

def switch_play_video(context, btnwid, oldval, newval, framecntwid, gstwid, frameimgwid):
    def _video_cap_loop():
        context.logger("Open GStreamer ... %s" % gstwid.value)
        try:
            cap = cv2.VideoCapture(gstwid.value, cv2.CAP_GSTREAMER)
            if cap.isOpened():
                context.logger('GStreamer is Opened!')
                count = 0
                while btnwid.value and count < framecntwid.value:
                    retval, frame = cap.read()
                    if retval:
                        frameimgwid.value = bytes(cv2.imencode(ext='.jpeg', img=frame)[1])
                    count += 1
                cap.release()
            btnwid.value = False
            context.logger('GStreamer is Stopped!')
        except Exception as err:
            context.logger('%s' % err)
    if newval:
        framecntwid.disabled = True
        btnwid.icon = 'stop-circle'
        btnwid.description = 'Stop'
        context.logger('Start Thread... ')
        threading.Thread(target=_video_cap_loop, args=()).start()
    else:
        framecntwid.disabled = False
        btnwid.icon = 'play'
        btnwid.description= 'Start'
        

def get_play_schema(gst):
     return {
        'type': 'page',
        'objs': [
            {
                'type': 'H',
                'objs': [
                    nbeasy_widget_togglebutton('switch_play', 'Start', icon='play'),
                    nbeasy_widget_int('frame_count', 'Frame Count', default=300, min_=100, max_=900, width=150),
                    nbeasy_widget_string('gst', 'GStreamer', gst, width='70%', readonly=True),
                ]
            },
            {
                'type': 'V',
                'objs': [
                    nbeasy_widget_image('frame_image', 'Image', '', format='jpeg')
                ],
                'align_items': 'center'
            },
            {
                'type': 'observe',
                'objs': [
                    {
                        'handler': switch_play_video,
                        'params': {
                            'source': 'switch_play',
                            'targets': [
                                'frame_count',
                                'gst',
                                'frame_image'
                            ]
                        }
                    },
                ]
            }
        ]
    }

# easy = nbeasy_schema_parse(get_play_schema(gst), debug=True);

## Data

In [5]:
import traitlets
import abc
import uuid
import datetime

In [6]:
class GSCom(traitlets.HasTraits):
    
    teename = traitlets.Unicode('')

    def __init__(self, parent, *args, **kwargs):
        super(GSCom, self).__init__(*args, **kwargs)
        if parent is not None:
            parent.children[self.name] = self
        self.parent = parent
        self.children = {}
            
    def gst_pipe(self):
        raise RuntimeError('abstract method called')
        
    def gst_str(self):
        gst = ' %s. ! queue ! ' % self.parent.teename if self.parent else ''
        gst += ' ! '.join(self.gst_pipe())
        for _, child in self.children.items():
            gst += child.gst_str()
        return gst 

### Data Source

In [7]:
class Camera(traitlets.HasTraits):
    device = traitlets.Int(min=0, max=1, default_value=0)
    uuid = traitlets.Unicode('')
    
    @traitlets.default('uuid')
    def _default_uuid(self):
        return uuid.UUID(int=uuid.getnode()).hex[-12:]


class DataSource(GSCom):
    teename = traitlets.Unicode('tee_raw')
    name = traitlets.Unicode()
    width = traitlets.Int(default_value=640)
    height = traitlets.Int(default_value=480)
    framerate = traitlets.Int(default_value=30)
    
    def __init__(self, *args, **kwargs):
        super(DataSource, self).__init__(None, *args, **kwargs)
   
    def gst_pipe(self):
        return [
            'tee name=%s' % self.teename
        ]
    

class USBCamera(Camera, DataSource):
    name = traitlets.Unicode(default_value='usbcam')
    device = traitlets.Int(default_value=1)
    
    def gst_pipe(self):
        raise NotImplementedError('usb cammera')
    

class CSICamera(Camera, DataSource):
    name = traitlets.Unicode(default_value='csicam')
    device = traitlets.Int(default_value=0)
    
    def gst_pipe(self):
        return [
            f'nvarguscamerasrc name={self.name} sensor-id={self.device}',
            f'"video/x-raw(memory:NVMM),width={self.width},height={self.height},format=(string)NV12,framerate={self.framerate}/1"',
            f'nvvidconv flip-method=6',
            f'"video/x-raw, width=(int){self.width}, height=(int){self.height}, format=(string)BGRx"',
            f'textoverlay halignment=0 valignment=2 text=CSI:{self.uuid}',
            f'clockoverlay halignment=2 time-format="%Y/%m/%d %H:%M:%S"',
        ] + super().gst_pipe()
    
gst_source_csi = CSICamera(device=0)
gst_source_csi.gst_str()

'nvarguscamerasrc name=csicam sensor-id=0 ! "video/x-raw(memory:NVMM),width=640,height=480,format=(string)NV12,framerate=30/1" ! nvvidconv flip-method=6 ! "video/x-raw, width=(int)640, height=(int)480, format=(string)BGRx" ! textoverlay halignment=0 valignment=2 text=CSI:48b02d5bcfa5 ! clockoverlay halignment=2 time-format="%Y/%m/%d %H:%M:%S" ! tee name=tee_raw'

### Data Convert

In [8]:
class DataConvert(GSCom):
    name = traitlets.Unicode(allow_none=False)

    def gst_pipe(self):
        return [
            ' tee name=%s ' % self.teename
        ]

    
class H264CodecCvt(DataConvert):
    teename = traitlets.Unicode('tee_h264')
    name = traitlets.Unicode('h264', read_only=True)
    bitrate = traitlets.Int(default_value=4000000)
    
    def gst_pipe(self):
        return [
            'nvvidconv',
            'nvv4l2h264enc bitrate=%d' % self.bitrate,
            'h264parse',
        ] + super().gst_pipe()
    
gst_cvt_h264 = H264CodecCvt(gst_source_csi, bitrate=4000000)    
gst_cvt_h264.gst_str()

' tee_raw. ! queue ! nvvidconv ! nvv4l2h264enc bitrate=4000000 ! h264parse !  tee name=tee_h264 '

### Data Sink

In [9]:
class DataSink(GSCom):
    name = traitlets.Unicode(allow_none=False)
   
  
class ShareMemorySink(DataSink):
    name = traitlets.Unicode(default_value='shmsink')
    path = traitlets.Unicode(default_value='/tmp/gst_socket.shm')
    size = traitlets.Int(default_value=67108864, min=0, max=4294967295)
    
    def __init__(self, parent, *args, **kwargs):
        super(ShareMemorySink, self).__init__(parent, *args, **kwargs)
        if os.path.exists(self.path):
            os.unlink(self.path)
    
    def gst_pipe(self):
        return [
            'videoconvert', 
            '"video/x-raw, format=(string)BGR"',
            f'shmsink socket-path="{self.path}" shm-size={self.size} sync=true wait-for-connection=false'
        ]
    

class MultiFilesSink(DataSink):
    name = traitlets.Unicode(default_value='multifilessink')
    duration = traitlets.Int(default_value=600, min=1, max=3600)
    maxfiles = traitlets.Int(default_value=200, min=1, max=1000)
    location = traitlets.Unicode(default_value=f'/tmp/{datetime.datetime.now().strftime("%Y%m%d")}')
    
    def __init__(self, parent, *args, **kwargs):
        super(MultiFilesSink, self).__init__(parent, *args, **kwargs)
        os.makedirs(self.location, exist_ok=True)
    
    def gst_pipe(self):
        return [
            f'splitmuxsink muxer=qtmux max-size-time={self.duration * 1000000000} max-files={self.maxfiles} location="{self.location}/%03d.mp4"',
        ]


class SRSRtmpSink(DataSink):
    name = traitlets.Unicode(default_value='rtmpsink')
    server = traitlets.Unicode(default_value='127.0.0.1')
    port = traitlets.Int(default_value=1935)
    stream = traitlets.Unicode(allow_none=False)
    
    def __init__(self, parent, *args, **kwargs):
        super(SRSRtmpSink, self).__init__(parent, *args, **kwargs)
        
    def gst_pipe(self):
        return [
            'flvmux',
            f'rtmpsink location=rtmp://{self.server}:{self.port}/live/{self.stream}',
        ]
    
gst_sink_shm = ShareMemorySink(parent=gst_source_csi, path='/tmp/gst_repnet.shm', size=20000000)
gst_sink_files = MultiFilesSink(parent=gst_cvt_h264, duration=300, maxfiles=10, location='/home/nano/omega/gst')
gst_sink_rtmp = SRSRtmpSink(parent=gst_cvt_h264, server='172.16.0.35', stream='48b02d5bcfa5')
gst_sink_shm.gst_str(), gst_sink_files.gst_str(), gst_sink_rtmp.gst_str()

(' tee_raw. ! queue ! videoconvert ! "video/x-raw, format=(string)BGR" ! shmsink socket-path="/tmp/gst_repnet.shm" shm-size=20000000 sync=true wait-for-connection=false',
 ' tee_h264. ! queue ! splitmuxsink muxer=qtmux max-size-time=300000000000 max-files=10 location="/home/nano/omega/gst/%03d.mp4"',
 ' tee_h264. ! queue ! flvmux ! rtmpsink location=rtmp://172.16.0.35:1935/live/48b02d5bcfa5')

In [10]:
gst_source_csi.gst_str()

'nvarguscamerasrc name=csicam sensor-id=0 ! "video/x-raw(memory:NVMM),width=640,height=480,format=(string)NV12,framerate=30/1" ! nvvidconv flip-method=6 ! "video/x-raw, width=(int)640, height=(int)480, format=(string)BGRx" ! textoverlay halignment=0 valignment=2 text=CSI:48b02d5bcfa5 ! clockoverlay halignment=2 time-format="%Y/%m/%d %H:%M:%S" ! tee name=tee_raw tee_raw. ! queue ! nvvidconv ! nvv4l2h264enc bitrate=4000000 ! h264parse !  tee name=tee_h264  tee_h264. ! queue ! splitmuxsink muxer=qtmux max-size-time=300000000000 max-files=10 location="/home/nano/omega/gst/%03d.mp4" tee_h264. ! queue ! flvmux ! rtmpsink location=rtmp://172.16.0.35:1935/live/48b02d5bcfa5 tee_raw. ! queue ! videoconvert ! "video/x-raw, format=(string)BGR" ! shmsink socket-path="/tmp/gst_repnet.shm" shm-size=20000000 sync=true wait-for-connection=false'

### Test

In [ ]:
%killall gst-launch

In [ ]:
%%bash --bg -s '{gst_source_csi.gst_str()}'

sudo rm -rf /tmp/gst_repnet.shm

bash -c "gst-launch-1.0 -e ${1}"

## JetRep

In [11]:
from multiprocessing import Queue, Process
from subprocess import Popen, PIPE, check_output, TimeoutExpired
from threading import Thread
import psutil
import queue

def get_pids(proname=None, cmdstr=None):
    if proname:
        try:
            return list(map(int, check_output(["pidof", proname]).split()))
        except:
            return []
    if keystr:
        pids = []
        for proc in psutil.process_iter():
            if cmdstr in ' '.join(proc.cmdline()):
                pids.append(proc.pid)
        return pids
    
# get_pids('gst-launch-1.0')

def kill_tree(pid, including_parent=True):    
    parent = psutil.Process(pid)
    children = parent.children(recursive=True)
    for child in children:
        child.kill()
    gone, still_alive = psutil.wait_procs(children, timeout=5)
    if including_parent:
        parent.kill()
        parent.wait(5)
 
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x, axis=None):
    e_x = np.exp(x - np.max(x, axis=axis, keepdims=True))
    return e_x / np.sum(e_x, axis=axis, keepdims=True)

### Inference Process

In [12]:
def repnet_inference_process(model_path, batch_queue, result_queue, status_queue, timeout):
    logger.debug('Start repnet_inference_process...')
    import tensorrt as trt
    import queue
    sys.path.append('/usr/src/tensorrt/samples/python')
    import common
    
    status_queue.put((100, 'starting'))
    try:
        with open(model_path, "rb") as f, \
            trt.Runtime(trt.Logger()) as runtime, \
            runtime.deserialize_cuda_engine(f.read()) as engine, \
            engine.create_execution_context() as context:

            inputs, outputs, bindings, stream = common.allocate_buffers(engine)

            status_queue.put((101, 'running'))
            while True:
                try:
                    token, np_frames = batch_queue.get(timeout=timeout)
                    if token < 0:
                        status_queue.put((0, 'quit'))
                        break
                except queue.Empty:
                    continue
                except Exception as err:
                    status_queue.put((-1, f'err: {err}'))
                    break

                np_frames = np_frames.astype(np.float32)
                np_frames -= 127.5
                np_frames /= 127.5
                inputs[0].host = np.reshape(np_frames, (1, 64, 112, 112, 3))

                trt_outputs = common.do_inference_v2(context, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)
                trt_outputs = [output.reshape(shape) for output, shape in zip(trt_outputs, ((-1, 1), (-1, 32)))]

                result_queue.put((token, trt_outputs[0].copy(), trt_outputs[1].copy()))
    except Exception as err:
        status_queue.put((-2, f'err: {err}'))

### Pre Worker

In [34]:
def prerep_process_worker(app):
    logger.debug("Start prerep_process_worker...")
    pre_frame = None
    keep_flag = 1
    while True:
        try:
            frame_bgr, fbat = app.read_put()
        except Exception as err:
            logger.error(f'Err: {err}')
            break
        fbat.raw_frames.append(frame_bgr)

        if app._black_box:
            black_x1, black_y1, black_x2, black_y2 = app._black_box
            frame_bgr[black_y1:black_y2, black_x1:black_x2, :] = 0

        if app._focus_box:
            focus_x1, focus_y1, focus_x2, focus_y2 = app._focus_box
            frame_bgr = frame_bgr[focus_y1:focus_y2, focus_x1:focus_x2, :]

        if app._area_thresh > 0:
            frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
            if pre_frame is not None:
                frame_tmp = cv2.absdiff(frame_gray, pre_frame)
                frame_tmp = cv2.threshold(frame_tmp, 20, 255, cv2.THRESH_BINARY)[1]
                frame_tmp = cv2.dilate(frame_tmp, None, iterations=2)
                contours, _ = cv2.findContours(frame_tmp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                if len(contours) > 0:
                    for contour in contours:
                        if cv2.contourArea(contour) > app._area_thresh:
                            keep_flag += 1
                            break
            pre_frame = frame_gray
        else:
            keep_flag += 1

        if keep_flag % app._stride == 0 or len(fbat.raw_frames) > app._max_raw_frames:
            frame_bgr = cv2.resize(frame_bgr, (112, 112))
            frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            fbat.selected_indices.append(len(fbat.raw_frames) - 1)
            fbat.bat_inputs.append(frame_rgb)
            if len(fbat.raw_frames) > app._max_raw_frames:
                diff = app.F - len(fbat.selected_indices) 
                for _ in range(diff):
                    fbat.selected_indices.append(len(fbat.raw_frames) - 1)
                    fbat.bat_inputs.append(frame_rgb)
                logger.warn(f'May be still frames is to many, diff[{diff}].')
            keep_flag = 1
    logger.debug('PreRep process worker end!')

### Post Worker

In [14]:
def postrep_process_worker(app):
    logger.debug("Start postrep_process_worker...")
    while True:
        try:
            fbat, within_scores, period_scores = app.read_get()
            if fbat is None:
                continue
        except queue.Empty:
            continue
        except Exception as err:
            logger.error(f'Err: {err}')
            break
            
        per_frame_periods = np.argmax(period_scores, axis=-1) + 1
        per_frame_counts = np.where(per_frame_periods < 3, 0.0, 1 / per_frame_periods)
        logger.debug(sum(per_frame_counts))
    
    logger.debug('PostRep process worker end!')

### Process Class

In [15]:
class FrameProcessApp(traitlets.HasTraits):
    
    frame_size = traitlets.Tuple(traitlets.Int(), traitlets.Int())
    focus_box = traitlets.List(trait=traitlets.Float(min=0., max=1.), default_value=[0.0, 0.1, 1.0, 1.0], minlen=4, maxlen=4)
    black_box = traitlets.List(trait=traitlets.Float(min=0., max=1.), default_value=[0.0, 0.0, 1.0, 1.0], minlen=4, maxlen=4)
    area_rate = traitlets.Float(default_value=0.002, min=0.0, max=0.05)
    strides = traitlets.List(trait=traitlets.Int(), default_value=[4], minlen=1, maxlen=3)
    
    F = 64
    K = F * 100
    
    def __init__(self, logger, read_gst, send_gst, mdata_in, mdata_out, *args, **kwargs):
        super(FrameProcessApp, self).__init__(*args, **kwargs)
        
        self.logger = logger
        self._focus_box = None
        self._black_box = None
        self._stride = 4
        self._max_raw_frames = self._stride * self.K
        self._area_thresh = 0
        self.frame_size = (640, 480)
        self.frame_rate = 30
        
        self._currbat = None
        self._batches = {}
        self.read_cap, self.read_gst = None, read_gst
        self.send_cap, self.send_gst = None, send_gst
        
        self.mdata_in = mdata_in
        self.mdata_out = mdata_out

        
    class FBatch(object):
        def __init__(self):
            self.token = int(time.time() * 1000)
            self.raw_frames = []
            self.bat_inputs = []
            self.selected_indices = []
        
    def make_fbatch(self):
        return self.FBatch()
        
    def open_reader(self):
        try:
            read_cap = cv2.VideoCapture(self.read_gst, cv2.CAP_GSTREAMER)
            if not read_cap.isOpened():
                raise RuntimeError('Could not open camera.')
            retval, _ = read_cap.read()
            if not retval:
                raise RuntimeError('Could not read image from camera.')
            
            self.frame_size = (
                int(read_cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                int(read_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            )
            self.frame_rate = int(read_cap.get(cv2.CAP_PROP_FPS))
                
        except:
            raise RuntimeError('Could not initialize camera.  Please see error trace.')
        self.read_cap = read_cap
        
    def close_reader(self):
        if self.read_cap:
            self.logger.debug("Exec cap release.")
            self.read_cap.release()
        self._currbat = None 
            
    def read_put(self):
        retval, frame = self.read_cap.read()
        if not retval:
            raise RuntimeError('Camera read error, maybe camera is released.')
        if self._currbat is None:
            self._currbat = self.make_fbatch()    
        if len(self._currbat.selected_indices) == self.F:
            self.mdata_in.put((self._currbat.token, np.asarray(self._currbat.bat_inputs)))
            self._batches[self._currbat.token] = self._currbat
            self._currbat = self.make_fbatch()
        return frame, self._currbat
        
        
    def read_get(self):
        token, within_scores, period_scores = self.mdata_out.get(timeout=1)
        jbat = None
        if token in self._batches:
            jbat = self._batches.pop(token)
        return jbat, within_scores, period_scores
    

    def open_sender(self):
        pass
    
    def close_sender(self):
        pass
    
    def write(self):
        pass
    
        
    @staticmethod
    def calc_rect_box(w, h, box):
        return [int(w * box[0]), int(h * box[1]), int(w * box[2]), int(h * box[3])]
        
    @traitlets.observe('frame_size')
    def _on_frame_size(self, change):   
        if self._focus_box:
            self._focus_box = self.calc_rect_box(change['new'][0], change['new'][1], self.focus_box)
        if self._black_box:
            self._black_box = self.calc_rect_box(change['new'][0], change['new'][1], self.black_box)
        if self._focus_box:
            x1, y1, x2, y2 = self._focus_box 
            self._area_thresh = int((x2 - x1) * (y2 - y1) * self.area_rate)
        else:
            self._area_thresh = int(change['new'][0] * change['new'][1] * self.area_rate)
        
    @traitlets.observe('focus_box')
    def _on_focus_box(self, change):
        if change['new'] == [0., 0., 1., 1.]:
            self._focus_box = None
            return
        self._focus_box = self.calc_rect_box(self.frame_size[0], self.frame_size[1], change['new'])
        
    @traitlets.observe('black_box')
    def _on_black_box(self, change):
        if change['new'] == [0., 0., 0., 0.] or change['new'] == [1., 1., 1., 1.]:
            self._black_box = None
            return
        self._black_box = self.calc_rect_box(self.frame_size[0], self.frame_size[1], change['new'])
        
    @traitlets.observe('area_rate')
    def _on_area_rate(self, change):
        if self._focus_box:
            x1, y1, x2, y2 = self._focus_box 
            self._area_thresh = (x2 - x1) * (y2 - y1) * change['new']
            return
        self._area_thresh = int(self.frame_size[0] * self.frame_size[1] * change['new'])
        
    @traitlets.observe('strides')
    def _on_strides(self, change):
        # TODO only support one stride
        self._stride = change['new'][0]
        self._max_raw_frames = self._stride * sefl.K
        
    @traitlets.validate('focus_box')
    def _check_focus_box(self, proposal):
        value = proposal['value']
        if value[0] >= value[2] or value[1] >= value[3]: 
            raise traitlets.TraitError(f'focus_box invalid: {value}')
        return value
                         
    @traitlets.validate('black_box')
    def _check_black_box(self, proposal):
        value = proposal['value']
        if value[0] > value[2] or value[1] > value[3]: 
            raise traitlets.TraitError(f'Parameter focus_box is invalid: {value}')
        return value
    
    def __str__(self):
        str_ = f'_area_thresh: {self._area_thresh} _focus_box: {self._focus_box} _black_box: {self._black_box} '
        str_ += f'_stride: {self._stride}'
        return str_

### Control Class

In [35]:
class JetRep(traitlets.HasTraits):
    gst_cmd = traitlets.Unicode('gst-launch-1.0', read_only=True)
    shm_path = traitlets.Unicode('/tmp/gst_repnet.shm', read_only=True)
    
    trt_weigth = traitlets.Unicode('/home/nano/models/repnet_b1.trt') 
    trt_msg_timeout = traitlets.Int(5)
    trycheck_count = traitlets.Int(6)
    
    def __init__(self, logger, *args, **kwargs):
        super(JetRep, self).__init__(*args, **kwargs)
        
        self.logger = logger
        self.gst_process = None
        self.engine_process = None
        self.pre_worker = None 
        self.post_worker = None
        self.msg_in = None
        self.msg_out = None
        self.msg_status = None
        
    def start_gst_launch(self):
        # TODO read config, make gst string
        gst_data_src = CSICamera(device=0, framerate=25)
        gst_h264_cvt = H264CodecCvt(gst_data_src, bitrate=4000000)    
        
        _ = ShareMemorySink(parent=gst_data_src, path=self.shm_path, size=20000000)
        _ = SRSRtmpSink(parent=gst_h264_cvt, server='172.16.0.35', stream='48b02d5bcfa5')
        _ = MultiFilesSink(parent=gst_h264_cvt, duration=300, maxfiles=10, location='/home/nano/omega/gst')
        
        # start
        # sudo systemctl restart nvargus-daemon
        self.gst_process = Popen(f'{self.gst_cmd} -e {gst_data_src.gst_str()}', stderr=PIPE, universal_newlines=True, shell=True, start_new_session=True)
        try:
            _, stderr = self.gst_process.communicate(timeout=3)
            raise RuntimeError('error: gst command line')
        except TimeoutExpired:
            pass
        
        self.logger.debug('Gst-launch started.')


    def stop_gst_launch(self):
        for pid in get_pids(self.gst_cmd):
            # os.killpg(os.getpgid(pid), signal.SIGINT) # start_new_session must be True
            kill_tree(os.getpgid(pid))
        self.gst_process = None


    def start_infer_engine(self):
        if not os.path.exists(self.trt_weigth):
            raise FileNotFoundError(f'error: {self.trt_weigth} is not found')
        self.engine_process = Process(target=repnet_inference_process, args=(
                self.trt_weigth, self.msg_in, self.msg_out, self.msg_status, self.trt_msg_timeout))
        self.engine_process.start()
        for i in range(1, self.trycheck_count + 1):
            try:
                errcode, msg = self.msg_status.get(timeout=self.trt_msg_timeout)
                self.logger.debug(f'start infer engine: {errcode}: {msg}')
                if errcode == 101:
                    break
            except queue.Empty:
                self.logger.debug(f'warning[{i}]: cannot start infer engine !!!')

    
    def stop_infer_engine(self):
        if self.engine_process:
            for i in range(1, self.trycheck_count + 1):
                self.msg_in.put((-1, None))
                try:
                    errcode, msg = self.msg_status.get(timeout=self.trt_msg_timeout)
                    self.logger.debug(f'stop infer engine: {errcode}: {msg}')
                    break
                except queue.Empty:
                    self.logger.debug(f'warning[{i}]: cannot stop infer engine !!!')
        self.engine_process = None

 
    def start_prerep_worker(self):
        self.app.open_reader()
        self.pre_worker = threading.Thread(target=prerep_process_worker, args=(self.app,), )
        self.pre_worker.start()

 
    def stop_prerep_worker(self):
        if self.pre_worker:
            self.app.close_reader()
            for i in range(1, self.trycheck_count + 1):
                if not self.pre_worker.is_alive():
                    self.pre_worker.join()
                    break
                time.sleep(0.01)
        self.pre_worker = None


    def start_postrep_worker(self):
        self.app.open_sender()
        self.post_worker = threading.Thread(target=postrep_process_worker, args=(self.app,))
        self.post_worker.start()

        
    def stop_postrep_worker(self):
        if self.post_worker:
            self.app.close_sender()
            for i in range(1, self.trycheck_count + 1):
                if not self.post_worker.is_alive():
                    self.post_worker.join()
                    break
                time.sleep(0.01)
        self.post_worker = None

        
    def start(self):
        self.stop()
        
        self.msg_in = Queue()
        self.msg_out = Queue()
        self.msg_status = Queue()
        
        self.start_gst_launch()
        self.start_infer_engine()
        
        read_gst_str = ' ! '.join([
            f'shmsrc socket-path={self.shm_path}',
            'video/x-raw, format=BGR, width=640, height=480',
            'appsink'
        ])
        send_gst_str = ''
        self.app = FrameProcessApp(self.logger, read_gst_str, send_gst_str, self.msg_in, self.msg_out)
        
        self.start_prerep_worker()
        self.start_postrep_worker()

    def stop(self):
        self.stop_prerep_worker()
        self.stop_postrep_worker()
        self.stop_infer_engine()
        self.stop_gst_launch()
        
        if self.msg_in: self.msg_in.close()
        if self.msg_out: self.msg_out.close()
        if self.msg_status: self.msg_status.close()

In [36]:
jr = JetRep(logger)

In [37]:
jr.start()

2021-11-05 22:04:10,967 - start_gst_launch:39 - JetRep - DEBUG - Gst-launch started.
2021-11-05 22:04:11,103 - repnet_inference_process:2 - JetRep - DEBUG - Start repnet_inference_process...
2021-11-05 22:04:14,769 - start_infer_engine:58 - JetRep - DEBUG - start infer engine: 100: starting
2021-11-05 22:04:19,775 - start_infer_engine:62 - JetRep - DEBUG - warning[2]: cannot start infer engine !!!
2021-11-05 22:04:24,781 - start_infer_engine:62 - JetRep - DEBUG - warning[3]: cannot start infer engine !!!
2021-11-05 22:04:24,852 - start_infer_engine:58 - JetRep - DEBUG - start infer engine: 101: running
2021-11-05 22:04:24,948 - prerep_process_worker:2 - JetRep - DEBUG - Start prerep_process_worker...
2021-11-05 22:04:24,949 - postrep_process_worker:2 - JetRep - DEBUG - Start postrep_process_worker...


In [38]:
print(jr.app)

_area_thresh: 614 _focus_box: None _black_box: None _stride: 4


In [26]:
jr.app._batches.keys()

dict_keys([])

In [41]:
len(jr.app._currbat.raw_frames)

149

2021-11-05 22:05:15,047 - postrep_process_worker:16 - JetRep - DEBUG - 1.8770833333333334


In [33]:
jr.pre_worker.is_alive(),  jr.post_worker.is_alive()

2021-11-05 22:03:28,930 - postrep_process_worker:11 - JetRep - ERROR - Err: handle is closed
2021-11-05 22:03:28,967 - postrep_process_worker:18 - JetRep - DEBUG - PostRep process worker end!


AttributeError: 'NoneType' object has no attribute 'is_alive'

In [ ]:
!pstree -p {jr.gst_process.pid}

In [ ]:
jr.app._currbat.selected_indices

In [ ]:
!pstree -p {jr.engine_process.pid}

In [24]:
IFrame(
    src='http://ossrs.net/players/rtc_player.html?vhost=__defaultVhost__&app=live&stream=48b02d5bcfa5&server=172.16.0.35&port=8181&autostart=true&schema=http',
    width='80%', height=680
)

## Repnet Model

### Pre Process

### Model Output

In [ ]:
import tensorrt as trt
from scipy.signal import medfilt

sys.path.append('/usr/src/tensorrt/samples/python')

import common

TRT_LOGGER = trt.Logger()

In [ ]:
np_frames = np_frames.astype(np.float32)
np_frames -= 127.5
np_frames /= 127.5

In [ ]:
MODEL_PATH = '/home/nano/models/repnet_b1.trt'

stride = STRIDES[0]


frames_seq_len = len(np_frames)
print("num_frames:", frames_seq_len)

num_batches = int(np.ceil(frames_seq_len / NUM_FRAMES / stride / BATCH_SIZE))
print("num_batches:", num_batches)

K = NUM_FRAMES * stride * BATCH_SIZE

output_shapes = [(-1, 1), (-1, NUM_FRAMES // 2)]
within_period_scores = []
period_scores = []
with open(MODEL_PATH, "rb") as f, \
    trt.Runtime(TRT_LOGGER) as runtime, \
    runtime.deserialize_cuda_engine(f.read()) as engine, \
    engine.create_execution_context() as context:
    inputs, outputs, bindings, stream = common.allocate_buffers(engine)
        
    start = time.time()
    for batch_idx in range(num_batches):
        idxes = range(batch_idx * K, (batch_idx + 1) * K, stride)
        idxes = np.clip(idxes, 0, frames_seq_len - 1)
        np_inputs = np.reshape(np.take(np_frames, idxes, axis=0), (BATCH_SIZE, NUM_FRAMES, INPUT_SIZE, INPUT_SIZE, 3))
        inputs[0].host = np_inputs
        trt_outputs = common.do_inference_v2(context, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)
        trt_outputs = [output.reshape(shape) for output, shape in zip(trt_outputs, output_shapes)]
        within_period_scores.append(trt_outputs[0].copy())
        period_scores.append(trt_outputs[1].copy())
    print(time.time() - start)
    
    np_within_period_scores = np.concatenate(within_period_scores, axis=0)
    np_period_scores = np.concatenate(period_scores, axis=0)
    
np_within_period_scores.shape, np_period_scores.shape

In [ ]:
a = []

### Post Process

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x, axis=None):
  e_x = np.exp(x - np.max(x, axis=axis, keepdims=True))
  return e_x / np.sum(e_x, axis=axis, keepdims=True)

#### Calculate Scores

In [ ]:
within_period_scores = sigmoid(np_within_period_scores)[:, 0]
within_period_scores[:10]

In [ ]:
per_frame_periods = np.argmax(np_period_scores, axis=-1) + 1
conf_pred_periods = np.max(softmax(np_period_scores, axis=-1), axis=-1)
per_frame_periods[:20], conf_pred_periods[:20]

In [ ]:
conf_pred_periods = np.where(per_frame_periods < 3, 0.0, conf_pred_periods)
conf_pred_periods[:10]

In [ ]:
within_period_scores = np.sqrt(within_period_scores * conf_pred_periods)
within_period_scores[:10], 'pred socre (%d): %.6f' % (stride, np.mean(within_period_scores))

In [ ]:
per_frame_counts = np.where(per_frame_periods < 3, 0.0, 1 / per_frame_periods)
per_frame_counts[:10]

In [ ]:
all_within_period_scores = np.repeat(within_period_scores, stride, axis=0)[:frames_seq_len]
all_period_scores = np.repeat(np_period_scores, stride, axis=0)[:frames_seq_len]
all_period_counts = (1 / stride) * np.repeat(per_frame_counts, stride, axis=0)[:frames_seq_len]
all_within_period_scores.shape, all_period_scores.shape

In [ ]:
sum(all_period_counts)

In [ ]:
all_within_period_binary = np.asarray(all_within_period_scores > 0.5)
all_within_period_binary.shape

In [ ]:
sum(all_period_counts * np.asarray(all_within_period_binary))

#### Calculate Count

In [ ]:
all_per_frame_periods = np.argmax(all_period_scores, axis=-1) + 1

per_frame_counts = np.where(np.less(all_per_frame_periods, 3), 0.0, np.divide(1.0,  stride * all_per_frame_periods))
per_frame_counts

## Make it together

In [ ]:
import threading, queue
import cv2, os, sys
import tensorrt as trt
from scipy.signal import medfilt
import numpy as np
import ipywidgets as widgets

sys.path.append('/usr/src/tensorrt/samples/python')

import common

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x, axis=None):
  e_x = np.exp(x - np.max(x, axis=axis, keepdims=True))
  return e_x / np.sum(e_x, axis=axis, keepdims=True)

BATCH_SIZE = 1
NUM_FRAMES = 64
INPUT_SIZE = 112
STRIDES = [4, 6]

TRT_LOGGER = trt.Logger()
from IPython.display import display, Markdown, HTML, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

usb_pipelines = [
    'v4l2src device=/dev/video1',
    'video/x-raw, width=(int)640, height=(int)480, framerate=(fraction)30/1',
    'videoconvert',
    'video/x-raw, format=(string)BGR',
    'videoflip video-direction=4',
    'textoverlay halignment=0 valignment=2 text=TalentAI',
    'clockoverlay halignment=2 time-format="%Y/%m/%d %H:%M:%S"',
    'appsink'
]

frames_queue = queue.Queue()

usb_gst = " ! ".join(usb_pipelines)
usb_gst

In [ ]:
def cal_rect_points(w, h, box):
    if box[0] < 1.0 and box[1] < 1.0 and box[2] <= 1.0 and box[3] <= 1.0:
        x1, y1 = int(w * box[0]), int(h * box[1])
        x2, y2 = int(w * box[2]), int(h * box[3])
    else:
        x1, y1 = box[0], box[1]
        x2, y2 = box[2], box[3]
    return x1, y1, x2, y2


In [ ]:
def repnet_grab_worker(gst, wid, frames_queue):
    cap = cv2.VideoCapture(gst, cv2.CAP_GSTREAMER)
    duration=500
    width=112
    height=112
    stride=4
    focus_box=[0.2, 0.2, 0.8, 0.8]
    black_box=[0.35, 0.0, 0.65,0.35]
    rm_still=True
    area_rate_thres=0.003
    
    f = int(cap.get(cv2.CAP_PROP_FPS))
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    if rm_still:
        pre_frame = None
        area_thres = area_rate_thres * w * h
        
    if black_box is not None:
        black_x1, black_y1, black_x2, black_y2 = cal_rect_points(w, h, black_box)
    if focus_box is not None:
        focus_x1, focus_y1, focus_x2, focus_y2 = cal_rect_points(w, h, focus_box)
        w = focus_x2 - focus_x1
        h = focus_y2 - focus_y1
    
    frames = []
    count = 0
    n_frames = f * duration
    keep_flag = 0
    try:
        while n_frames > 0:
            retval, frame_bgr = cap.read()
            n_frames -= 1
            if not retval:
                continue

            if black_box is not None:
                frame_bgr[black_y1:black_y2, black_x1:black_x2, :] = 0

            if wid:
                if focus_box is not None:
                    cv2.rectangle(frame_bgr, (focus_x1, focus_y1), (focus_x2, focus_y2), (0, 255, 0), 2)

                wid.value = bytes(cv2.imencode(ext='.jpeg', img=frame_bgr)[1])

            if focus_box is not None:
                frame_bgr = frame_bgr[focus_y1:focus_y2, focus_x1:focus_x2, :]


            if rm_still:
                frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
                if pre_frame is not None:
                    frame_tmp = cv2.absdiff(frame_gray, pre_frame)
                    frame_tmp = cv2.threshold(frame_tmp, 20, 255, cv2.THRESH_BINARY)[1]
                    frame_tmp = cv2.dilate(frame_tmp, None, iterations=2)
                    contours, _ = cv2.findContours(frame_tmp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                    if len(contours) > 0:
                        for contour in contours:
                            if cv2.contourArea(contour) > area_thres:
                                keep_flag += 1
                                break
                pre_frame = frame_gray

            frame_bgr = cv2.resize(frame_bgr, (width, height))
            frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            if not rm_still or keep_flag % stride == 0:
                keep_flag = 0
                frames.append(frame_rgb)
                if len(frames) == 64 and frames_queue:
                    frames_queue.put(frames)
                    frames = []
    finally:
        cap.release()

In [ ]:
def repnet_model(wid, frames_queue):
    count = 0
    wid.value = "Count: 0"
    output_shapes = [(-1, 1), (-1, NUM_FRAMES // 2)]
    MODEL_PATH = '/home/nano/models/repnet_b1.trt'
    i = 0
    with open(MODEL_PATH, "rb") as f, \
        trt.Runtime(TRT_LOGGER) as runtime, \
        runtime.deserialize_cuda_engine(f.read()) as engine, \
        engine.create_execution_context() as context:
            
        inputs, outputs, bindings, stream = common.allocate_buffers(engine)
        
        while True:
            np_frames = np.asarray(frames_queue.get(), dtype=np.float32) 
            np_frames = np_frames.astype(np.float32)
            np_frames -= 127.5
            np_frames /= 127.5
            
            np_inputs = np.reshape(np_frames, (1, NUM_FRAMES, INPUT_SIZE, INPUT_SIZE, 3))

            within_period_scores = []
            period_scores = []
            inputs[0].host = np_inputs
            trt_outputs = common.do_inference_v2(context, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)
            trt_outputs = [output.reshape(shape) for output, shape in zip(trt_outputs, output_shapes)]
            np_within_period_scores = trt_outputs[0].copy()
            np_period_scores = trt_outputs[1].copy()
            
            
            per_frame_periods = np.argmax(np_period_scores, axis=-1) + 1
            conf_pred_periods = np.max(softmax(np_period_scores, axis=-1), axis=-1)
            conf_pred_periods = np.where(per_frame_periods < 3, 0.0, conf_pred_periods)
            
            within_period_scores = sigmoid(np_within_period_scores)[:, 0]
            within_period_scores = np.sqrt(within_period_scores * conf_pred_periods)
            within_period_binary = np.asarray(within_period_scores > 0.5)

            per_frame_counts = np.where(per_frame_periods < 3, 0.0, 1 / per_frame_periods)
            count += sum(per_frame_counts)
            i += 1
            
            wid.value = "Count: %d" % count
            if i > 20:
                break

In [ ]:
count_vid = widgets.Text(value="Count: 0")
grap_wid = widgets.Image(format='jpeg')
display(widgets.VBox([count_vid, grap_wid]))

repnet_grap_worker = threading.Thread(target=repnet_grab_worker, args=(usb_gst, grap_wid, frames_queue))
repnet_grap_worker.start()
repnet_model(count_vid, frames_queue)
repnet_grap_worker.join()

## References

- [multiprocessing fork() vs spawn()][1] 
<div class="alert alert-info"><code>
import multiprocessing as mp
mp.set_start_method('spawn', force=True)
mp.set_start_method('fork', force=True)
</code></div>

[1]: https://stackoverflow.com/questions/64095876/multiprocessing-fork-vs-spawn

## FAQs

- [Failed to create CaptureSession][1]
<div class="alert alert-warning">
This may happen if there is another session using this same camera (hung or alive) and try to use same camera again; if this issue is happening sometimes. You can close all sessions and reset the nvargus-daemon to reset the argus framework and try nvgstcapture-1.0 again. 
<code>
sudo systemctl restart nvargus-daemon
</code></div>

[1]: https://forums.developer.nvidia.com/t/error-generated-gstnvarguscamerasrc-cpp-execute-543-failed-to-create-capturesession/112431